In [60]:
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,100).__str__()
import cv2
import math
from glob import glob
import numpy as np
import pandas as pd
import cv2
from tqdm.notebook import tqdm
import openslide
import libtiff
from shapely.geometry import Polygon
import lxml.etree as ET
from kneed import KneeLocator
from scipy.interpolate import splprep, splev
from typing import List, Dict

libtiff.libtiff_ctypes.suppress_warnings()

In [55]:
tissue_kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
tissue_kernel_2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

def get_otsu_mask(_img):
    _, _, v = cv2.split(cv2.cvtColor(_img, cv2.COLOR_RGB2YUV))
    v = cv2.medianBlur(v, 5)
    _, _mask = cv2.threshold(v, -1, 255, cv2.THRESH_OTSU)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=5)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=5)
    return _mask

def get_gray_mask(_img):
    v = cv2.cvtColor(_img, cv2.COLOR_RGB2GRAY)
    v = cv2.medianBlur(v, 5)
    _, _mask = cv2.threshold(v, -1, 255, cv2.THRESH_OTSU)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=5)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=5)
    _mask = ~_mask
    return _mask

def get_level(n, cnt):
    if (n % 2) == 0:
        return get_level((n // 2), cnt+1)
    else:
        return cnt

def get_mask(slide_name, magnification):
    slide_path = glob(f'/workspace/data3/A100_data/*/*/{slide_name}.svs')
    if len(slide_path) == 0:
        slide_path = glob(f'/workspace/data4/A100_data/*/*/{slide_name}.svs')
        if len(slide_path) == 0:
            slide_path = glob(f'/workspace/data3/A100_data/*/{slide_name}.svs')
    slide_path = slide_path[0]
    
    wsi_slide = openslide.OpenSlide(slide_path)
    mpp = wsi_slide.properties['aperio.MPP']
    if float(mpp) < 0.40:
        ORG_MAG = 40
    else:
        ORG_MAG = 20
    
    LEVEL = get_level(int(ORG_MAG // 1.25), 0)
    w_, h_ = wsi_slide.level_dimensions[LEVEL//2]
    wsi_array = np.array(wsi_slide.read_region([0, 0], LEVEL//2, [w_, h_]))
    if LEVEL % 2:
        wsi_array = cv2.resize(wsi_array, (w_//2, h_//2))
    otsu_mask = get_otsu_mask(wsi_array)
    gray_mask = get_gray_mask(wsi_array)
    tissue_mask = cv2.bitwise_or(otsu_mask, gray_mask)
    
    LEVEL = get_level(int(ORG_MAG // magnification), 0)
    w_, h_ = wsi_slide.level_dimensions[LEVEL//2]
    if LEVEL % 2:
        w_, h_ = w_//2, h_//2
    # tissue_mask = cv2.resize(tissue_mask, (w_, h_))
    
    return tissue_mask, mpp, LEVEL

def calculate_contour_distance(contour1, contour2): 
    (x1, y1), r1 = cv2.minEnclosingCircle(contour1)
    (x2, y2), r2 = cv2.minEnclosingCircle(contour2)
    a = x2 - x1
    b = y2 - y1
    return math.sqrt((a * a) + (b * b))

def merge_contours(contour1, contour2):
    return np.concatenate((contour1, contour2), axis=0)

def agglomerative_cluster(contours, threshold_distance=500.0):
    current_contours = contours
    while len(current_contours) > 1:
        min_distance = None
        min_coordinate = None

        for x in range(len(current_contours)-1):
            for y in range(x+1, len(current_contours)):
                distance = calculate_contour_distance(current_contours[x], current_contours[y])
                if min_distance is None:
                    min_distance = distance
                    min_coordinate = (x, y)
                elif distance < min_distance:
                    min_distance = distance
                    min_coordinate = (x, y)

        if min_distance < threshold_distance:
            index1, index2 = min_coordinate
            current_contours[index1] = merge_contours(current_contours[index1], current_contours[index2])
            del current_contours[index2]
        else: 
            break

    return current_contours

def resize_image(image, magnification):
    height, width, _ = image.shape
    new_size = (width // int(magnification * 0.8), height // int(magnification * 0.8))
    return cv2.resize(image, new_size, cv2.INTER_MAX)

def get_points_list(contour_points, level, mag):
    downsample = 2 ** (level + 3)
    offset = {'X': 0, 'Y': 0}
    points_list = []
    for contour in contour_points:
        points = []
        for point in contour:
            x = point[0][0] * downsample + offset['X']
            y = point[0][1] * downsample + offset['Y']
            points.append({'X': x, 'Y': y})
        x = contour[0][0][0] * downsample + offset['X']
        y = contour[0][0][1] * downsample + offset['Y']
        points.append({'X': x, 'Y': y})
        points_list.append(points)
    return points_list

def get_hull_points_list(hull_lp, level, mag):
    downsample = 2**(level + 3)
    offset={'X': 0,'Y': 0}
    pointsList = []
    for j in range(np.shape(hull_lp)[0]):
        pointList = []
        for i in range(np.shape(hull_lp[j])[0]):
            point = {'X': (hull_lp[j][i][0][0] * downsample) + offset['X'], 'Y': (hull_lp[j][i][0][1] * downsample) + offset['Y']}
            pointList.append(point)
        point = {'X': (hull_lp[j][0][0][0] * downsample) + offset['X'], 'Y': (hull_lp[j][0][0][1] * downsample) + offset['Y']}
        pointList.append(point)
        pointsList.append(pointList)
    return pointsList

def make_hull(new_maskPoints_l):
    hull_l = []
    for i in range(len(new_maskPoints_l)):
        hull = cv2.convexHull(new_maskPoints_l[i])
        hull_l.append(hull)

    hull_lp = []
    for i in range(len(hull_l)):
        x,y = hull_l[i].T
        x = x.tolist()[0]
        y = y.tolist()[0]
        tck, u = splprep([x,y], u=None, s=1.0, per=1)
        u_new = np.linspace(u.min(), u.max(), 150)
        x_new, y_new = splev(u_new, tck, der=0)
        res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new,y_new)]
        smooth = np.asarray(res_array, dtype=np.int32)
        hull_lp.append(smooth)
    return hull_lp

In [73]:
def to_xml(slide_name = 'A100_00206_03_stm_c1', mag = 10, thres = 0.3, thresdist = 4):
    tissue_mask, mpp, level = get_mask(slide_name, mag)

    read_path = '/workspace/data4/changwoo/SDP/mask/Carcinoma_v4/'+str(mag)+'X/'+f'{slide_name}.png'
    img = cv2.imread(read_path)
    
    img = resize_image(img, mag)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.bitwise_and(img_gray, tissue_mask)
    img_blur = cv2.GaussianBlur(img_gray, (0, 0), 5)

    thres1 = int(thres * 255)
    _, mask = cv2.threshold(img_blur, thres1, 255, cv2.THRESH_BINARY_INV)
    maskPoints, contours = cv2.findContours(np.uint8(mask), mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_NONE)
    maskPoints_l = sorted(maskPoints, key = cv2.contourArea, reverse = True)
    maskPoints_l = maskPoints_l[1:]
    
    pointsList = get_points_list(maskPoints_l, level, mag)

    Annotations = ET.Element('Annotations')
    # create new xml Tree - Annotations
    name1 = 'threshold '+str(thres)
    Annotations = ET.Element('Annotations', attrib={'MicronsPerPixel': mpp})
    Annotation = ET.SubElement(Annotations, 'Annotation', attrib={'Id': '1', 'Name': name1, 'ReadOnly': '0', 'NameReadOnly': '0', 'LineColorReadOnly': '0', 'Incremental': '0', 'Type': '4', 'LineColor': '6640479', 'Visible': '1', 'Selected': '1', 'MarkupImagePath': '', 'MacroName': ''})
    Regions = ET.SubElement(Annotation, 'Regions')
    for i in range(1, len(pointsList)+1):
        regionID = i
        Region = ET.SubElement(Regions, 'Region', attrib={'Id': str(regionID), 'Type': '0', 'NegativeROA': '0', 'InputRegionId': '0', 'Analyze': '1', 'DisplayId': str(regionID)})
        Vertices = ET.SubElement(Region, 'Vertices')
        for point in pointsList[i-1]: # add new Vertex
            ET.SubElement(Vertices, 'Vertex', attrib={'X': str(point['X']), 'Y': str(point['Y']), 'Z': '0'})

    distance = thresdist * 125
    new_maskPoints_l = agglomerative_cluster(maskPoints_l, threshold_distance = distance)
    
    hull_lp = make_hull(new_maskPoints_l)
    pointsList = get_hull_points_list(hull_lp, level, mag)

    # create new xml Tree - Annotations
    name2 = 'threshold distance ' + str(thresdist) + 'mm'
    Annotation = ET.SubElement(Annotations, 'Annotation', attrib={'Id': '2', 'Name': name2, 'ReadOnly': '0', 'NameReadOnly': '0', 'LineColorReadOnly': '0', 'Incremental': '0', 'Type': '4', 'LineColor': '65535', 'Visible': '1', 'Selected': '1', 'MarkupImagePath': '', 'MacroName': ''})
    Regions = ET.SubElement(Annotation, 'Regions')
    for i in range(1, len(pointsList)+1):
        regionID = i
        Region = ET.SubElement(Regions, 'Region', attrib={'Id': str(regionID), 'Type': '0', 'NegativeROA': '0', 'InputRegionId': '0', 'Analyze': '1', 'DisplayId': str(regionID)})
        Vertices = ET.SubElement(Region, 'Vertices')
        for point in pointsList[i-1]: # add new Vertex
            ET.SubElement(Vertices, 'Vertex', attrib={'X': str(point['X']), 'Y': str(point['Y']), 'Z': '0'})

    xml_data = ET.tostring(Annotations, pretty_print=True)

    save_path = '/workspace/src/chaehyeon/png_contour/left/'+f'{slide_name}.xml'
    with open(save_path, 'w') as f:
        f.write(xml_data.decode())

In [66]:
df = pd.read_csv('/workspace/src/chaehyeon/png_contour/Carcinoma_v4_10X_TP.csv')
df = df.reset_index(drop=True)
name_list = df['name'].tolist()

In [67]:
in_list = ['A100_02127_10_gu_c1.xml', 'A100_00189_01_stm_c1.xml', 'A100_00277_01_stm_c1.xml', 'A100_00568_05_col_c1.xml', 'A100_00875_07_col_c1.xml', 'A100_01571_01_gu_c1.xml', 'A100_02011_08_gu_c1.xml', 'A100_01863_05_en_c1.xml', 'A100_01885_03_en_c1.xml', 'A100_01374_09_col_c1.xml', 'A100_01735_02_en_c1.xml', 'A100_01414_03_en_c1.xml', 'A100_01793_01_en_c1.xml', 'A100_00527_07_lvr_c1.xml', 'A100_01602_07_gu_c1.xml', 'A100_02470_07_stm_c1.xml', 'A100_01548_02_gu_c1.xml', 'A100_02112_01_gu_c1.xml', 'A100_01202_07_en_c1.xml', 'A100_01171_01_gu_c1.xml', 'A100_01813_04_en_c1.xml', 'A100_01275_23_stm_c1.xml', 'A100_00875_11_col_c1.xml', 'A100_02413_02_brs_c1.xml', 'A100_02099_10_gu_c1.xml', 'A100_01602_09_gu_c1.xml', 'A100_02926_07_col_c1.xml', 'A100_01749_01_en_c1.xml', 'A100_02524_08_brs_c1.xml', 'A100_00405_01_stm_c1.xml', 'A100_01517_08_stm_c1.xml', 'A100_00315_01_stm_c1.xml', 'A100_00082_01_stm_c1.xml', 'A100_01871_03_en_c1.xml', 'A100_00944_07_en_c1.xml', 'A100_01831_01_en_c1.xml', 'A100_01159_10_gu_c1.xml', 'A100_03475_05_col_c1.xml', 'A100_02065_10_gu_c1.xml', 'A100_02563_03_col_c1.xml', 'A100_01863_04_en_c1.xml', 'A100_00156_01_stm_c1.xml', 'A100_02096_02_gu_c1.xml', 'A100_01147_07_gu_c1.xml', 'A100_02575_03_col_c1.xml', 'A100_01751_02_en_c1.xml', 'A100_00945_03_col_c1.xml', 'A100_01820_03_en_c1.xml', 'A100_00881_08_col_c1.xml', 'A100_00785_03_fem_c1.xml', 'A100_01159_03_gu_c1.xml', 'A100_02096_05_gu_c1.xml', 'A100_01674_01_en_c1.xml', 'A100_02280_05_stm_c1.xml', 'A100_03453_02_col_c1.xml', 'A100_03478_01_col_c1.xml', 'A100_02401_18_brs_c1.xml', 'A100_02301_11_stm_c1.xml', 'A100_01014_05_en_c1.xml', 'A100_01498_07_gu_c1.xml', 'A100_01006_07_col_c1.xml', 'A100_02524_11_brs_c1.xml', 'A100_02339_01_stm_c1.xml', 'A100_01149_01_gu_c1.xml', 'A100_00945_04_col_c1.xml', 'A100_02152_10_gu_c1.xml', 'A100_02953_08_col_c1.xml', 'A100_02524_19_brs_c1.xml', 'A100_01200_03_en_c1.xml', 'A100_00881_03_col_c1.xml', 'A100_00840_04_col_c1.xml', 'A100_00475_03_brs_c1.xml', 'A100_01176_03_gu_c1.xml', 'A100_01192_20_gu_c1.xml', 'A100_00475_01_brs_c1.xml', 'A100_02386_02_stm_c1.xml', 'A100_02413_14_brs_c1.xml', 'A100_00761_03_fem_c1.xml', 'A100_00530_06_lvr_c1.xml', 'A100_01575_02_gu_c1.xml', 'A100_02307_12_stm_c1.xml', 'A100_01564_04_gu_c1.xml', 'A100_02098_03_gu_c1.xml', 'A100_00386_01_stm_c1.xml', 'A100_01192_19_gu_c1.xml', 'A100_01600_05_gu_c1.xml', 'A100_00516_01_brs_c1.xml', 'A100_00210_13_stm_c1.xml', 'A100_02926_06_col_c1.xml', 'A100_01642_08_stm_c1.xml', 'A100_02413_15_brs_c1.xml', 'A100_01642_30_stm_c1.xml', 'A100_00384_18_stm_c1.xml', 'A100_01643_13_stm_c1.xml', 'A100_02156_05_stm_c1.xml', 'A100_02360_04_stm_c1.xml', 'A100_02467_10_brs_c1.xml', 'A100_02360_12_stm_c1.xml', 'A100_00206_03_stm_c1.xml', 'A100_01575_09_gu_c1.xml', 'A100_00527_10_lvr_c1.xml', 'A100_02383_06_stm_c1.xml', 'A100_01642_09_stm_c1.xml', 'A100_01861_01_en_c1.xml', 'A100_01080_01_hn_c1.xml', 'A100_02408_01_brs_c1.xml', 'A100_02006_08_gu_c1.xml', 'A100_01863_07_en_c1.xml', 'A100_02401_20_brs_c1.xml', 'A100_02301_03_stm_c1.xml', 'A100_00568_02_col_c1.xml', 'A100_02524_12_brs_c1.xml', 'A100_01731_01_en_c1.xml', 'A100_02127_02_gu_c1.xml', 'A100_02098_05_gu_c1.xml', 'A100_00555_03_col_c1.xml', 'A100_02126_01_gu_c1.xml', 'A100_00583_03_col_c1.xml', 'A100_02455_13_bp_c1.xml', 'A100_00151_01_stm_c1.xml', 'A100_00938_04_en_c1.xml', 'A100_02563_05_col_c1.xml', 'A100_00347_01_stm_c1.xml', 'A100_00785_01_fem_c1.xml', 'A100_02383_03_stm_c1.xml', 'A100_02572_03_col_c1.xml', 'A100_01205_19_en_c1.xml', 'A100_00517_07_lvr_c1.xml', 'A100_02877_05_col_c1.xml', 'A100_00540_01_lvr_c1.xml', 'A100_02401_23_brs_c1.xml', 'A100_02156_04_stm_c1.xml', 'A100_01560_04_gu_c1.xml', 'A100_02070_02_gu_c1.xml', 'A100_02336_03_stm_c1.xml', 'A100_02520_02_brs_c1.xml', 'A100_01202_06_en_c1.xml', 'A100_03478_06_col_c1.xml', 'A100_02328_02_stm_c1.xml', 'A100_01795_03_en_c1.xml', 'A100_02413_19_brs_c1.xml', 'A100_01374_08_col_c1.xml', 'A100_01571_03_gu_c1.xml', 'A100_02152_06_gu_c1.xml', 'A100_02328_01_stm_c1.xml', 'A100_00925_04_gu_c1.xml', 'A100_00751_11_fem_c1.xml', 'A100_00206_05_stm_c1.xml', 'A100_01846_02_en_c1.xml', 'A100_02307_08_stm_c1.xml', 'A100_02114_09_gu_c1.xml', 'A100_02314_06_stm_c1.xml', 'A100_02483_05_bp_c1.xml', 'A100_03475_02_col_c1.xml', 'A100_00152_01_stm_c1.xml', 'A100_01560_06_gu_c1.xml', 'A100_01653_14_stm_c1.xml', 'A100_00209_07_stm_c1.xml', 'A100_01863_08_en_c1.xml', 'A100_02054_04_gu_c1.xml', 'A100_00935_08_en_c1.xml', 'A100_00126_01_stm_c1.xml', 'A100_01101_01_gu_c1.xml', 'A100_01813_01_en_c1.xml', 'A100_01176_05_gu_c1.xml', 'A100_02356_03_stm_c1.xml', 'A100_00209_09_stm_c1.xml', 'A100_02479_05_bp_c1.xml', 'A100_00742_02_col_c1.xml', 'A100_00929_05_en_c1.xml', 'A100_01527_13_stm_c1.xml', 'A100_01080_02_hn_c1.xml', 'A100_01579_11_gu_c1.xml', 'A100_02070_07_gu_c1.xml', 'A100_02385_12_stm_c1.xml', 'A100_01653_18_stm_c1.xml', 'A100_01159_14_gu_c1.xml', 'A100_01192_17_gu_c1.xml', 'A100_00944_05_en_c1.xml', 'A100_00581_07_col_c1.xml', 'A100_02558_03_col_c1.xml', 'A100_00881_06_col_c1.xml', 'A100_00318_06_stm_c1.xml', 'A100_00875_08_col_c1.xml', 'A100_00385_01_stm_c1.xml', 'A100_00285_06_stm_c1.xml', 'A100_02158_01_stm_c1.xml', 'A100_01159_04_gu_c1.xml', 'A100_01812_01_en_c1.xml', 'A100_01159_19_gu_c1.xml', 'A100_01555_08_gu_c1.xml', 'A100_01123_01_gu_c1.xml', 'A100_01831_02_en_c1.xml', 'A100_03407_01_col_c1.xml', 'A100_01896_02_en_c1.xml', 'A100_02099_11_gu_c1.xml', 'A100_01813_02_en_c1.xml', 'A100_01644_10_stm_c1.xml', 'A100_01049_13_bp_c1.xml', 'A100_02096_03_gu_c1.xml', 'A100_02120_07_gu_c1.xml', 'A100_02055_05_gu_c1.xml', 'A100_02139_07_gu_c1.xml', 'A100_00001_01_stm_c1.xml', 'A100_00725_01_col_c1.xml', 'A100_01608_05_gu_c1.xml', 'A100_01842_01_en_c1.xml', 'A100_00277_04_stm_c1.xml', 'A100_02400_01_brs_c1.xml', 'A100_01863_06_en_c1.xml', 'A100_02524_04_brs_c1.xml', 'A100_02475_04_bp_c1.xml', 'A100_02918_06_col_c1.xml', 'A100_03451_01_col_c1.xml', 'A100_01662_17_stm_c1.xml', 'A100_03485_01_col_c1.xml', 'A100_00537_06_lvr_c1.xml', 'A100_01044_04_bp_c1.xml', 'A100_00840_06_col_c1.xml', 'A100_01517_11_stm_c1.xml', 'A100_02026_05_gu_c1.xml', 'A100_02112_03_gu_c1.xml', 'A100_01737_02_en_c1.xml', 'A100_00202_03_stm_c1.xml', 'A100_01046_09_bp_c1.xml', 'A100_02098_04_gu_c1.xml', 'A100_00530_08_lvr_c1.xml', 'A100_01270_08_stm_c1.xml', 'A100_00875_09_col_c1.xml', 'A100_01908_02_en_c1.xml', 'A100_00846_07_en_c1.xml', 'A100_01031_22_stm_c1.xml', 'A100_02307_09_stm_c1.xml', 'A100_01232_03_en_c1.xml', 'A100_02006_10_gu_c1.xml', 'A100_02918_08_col_c1.xml', 'A100_00057_01_stm_c1.xml', 'A100_00384_20_stm_c1.xml', 'A100_02301_04_stm_c1.xml', 'A100_02350_01_stm_c1.xml', 'A100_01642_47_stm_c1.xml', 'A100_02011_18_gu_c1.xml', 'A100_02099_07_gu_c1.xml', 'A100_01560_03_gu_c1.xml', 'A100_00769_01_fem_c1.xml', 'A100_01708_02_en_c1.xml', 'A100_02538_08_col_c1.xml', 'A100_00249_03_stm_c1.xml', 'A100_02401_06_brs_c1.xml', 'A100_02524_27_brs_c1.xml', 'A100_02314_08_stm_c1.xml', 'A100_01498_04_gu_c1.xml', 'A100_03456_02_col_c1.xml', 'A100_02524_24_brs_c1.xml', 'A100_02011_04_gu_c1.xml', 'A100_02296_01_stm_c1.xml', 'A100_00755_02_fem_c1.xml', 'A100_02467_04_brs_c1.xml', 'A100_01159_05_gu_c1.xml', 'A100_02527_04_brs_c1.xml', 'A100_01642_22_stm_c1.xml', 'A100_01299_16_stm_c1.xml', 'A100_00217_01_stm_c1.xml', 'A100_02317_13_stm_c1.xml', 'A100_00934_25_sk_c1.xml', 'A100_02156_02_stm_c1.xml', 'A100_01950_14_stm_c1.xml', 'A100_00318_13_stm_c1.xml', 'A100_03451_02_col_c1.xml', 'A100_01186_14_gu_c1.xml', 'A100_01642_33_stm_c1.xml', 'A100_01600_08_gu_c1.xml', 'A100_02467_14_brs_c1.xml', 'A100_01767_02_en_c1.xml', 'A100_02099_12_gu_c1.xml', 'A100_01908_01_en_c1.xml', 'A100_02467_02_brs_c1.xml', 'A100_02877_03_col_c1.xml', 'A100_00199_07_stm_c1.xml', 'A100_03442_05_col_c1.xml', 'A100_00262_10_stm_c1.xml', 'A100_00517_09_lvr_c1.xml', 'A100_02120_10_gu_c1.xml', 'A100_01544_06_gu_c1.xml', 'A100_01598_01_gu_c1.xml', 'A100_01498_08_gu_c1.xml', 'A100_00575_06_col_c1.xml', 'A100_00517_10_lvr_c1.xml', 'A100_02054_03_gu_c1.xml', 'A100_01107_01_gu_c1.xml', 'A100_02558_06_col_c1.xml', 'A100_03453_03_col_c1.xml', 'A100_01176_02_gu_c1.xml', 'A100_02054_09_gu_c1.xml', 'A100_00522_02_lvr_c1.xml', 'A100_03457_03_col_c1.xml', 'A100_02524_15_brs_c1.xml', 'A100_01004_07_en_c1.xml', 'A100_01420_06_col_c1.xml', 'A100_01941_11_stm_c1.xml', 'A100_01186_12_gu_c1.xml', 'A100_01281_28_stm_c1.xml', 'A100_02011_07_gu_c1.xml', 'A100_02317_08_stm_c1.xml', 'A100_02492_06_brs_c1.xml', 'A100_01374_06_col_c1.xml', 'A100_01773_02_en_c1.xml', 'A100_02127_13_gu_c1.xml', 'A100_01244_02_en_c1.xml', 'A100_02296_04_stm_c1.xml', 'A100_02112_04_gu_c1.xml', 'A100_02120_01_gu_c1.xml', 'A100_02877_02_col_c1.xml', 'A100_01579_04_gu_c1.xml', 'A100_01642_35_stm_c1.xml', 'A100_02317_07_stm_c1.xml', 'A100_02360_13_stm_c1.xml', 'A100_01616_05_gu_c1.xml', 'A100_02386_04_stm_c1.xml', 'A100_01926_17_stm_c1.xml', 'A100_00205_02_stm_c1.xml', 'A100_01842_06_en_c1.xml', 'A100_00515_01_brs_c1.xml', 'A100_01159_09_gu_c1.xml', 'A100_02558_04_col_c1.xml', 'A100_02385_05_stm_c1.xml', 'A100_00840_05_col_c1.xml', 'A100_03478_05_col_c1.xml', 'A100_00202_01_stm_c1.xml', 'A100_02365_02_stm_c1.xml', 'A100_01774_04_en_c1.xml', 'A100_01735_04_en_c1.xml', 'A100_01896_01_en_c1.xml', 'A100_01486_12_en_c1.xml', 'A100_01555_03_gu_c1.xml', 'A100_01374_10_col_c1.xml', 'A100_01766_05_en_c1.xml', 'A100_02068_03_gu_c1.xml', 'A100_00935_10_en_c1.xml', 'A100_02307_14_stm_c1.xml', 'A100_01686_01_en_c1.xml', 'A100_02026_02_gu_c1.xml', 'A100_00535_08_lvr_c1.xml', 'A100_01560_08_gu_c1.xml', 'A100_01555_06_gu_c1.xml', 'A100_02386_09_stm_c1.xml', 'A100_02401_22_brs_c1.xml', 'A100_02120_02_gu_c1.xml', 'A100_01544_08_gu_c1.xml', 'A100_01090_02_sk_c1.xml', 'A100_00318_11_stm_c1.xml', 'A100_00309_05_stm_c1.xml', 'A100_02413_12_brs_c1.xml', 'A100_02116_02_gu_c1.xml', 'A100_02011_12_gu_c1.xml', 'A100_00591_06_col_c1.xml', 'A100_01517_12_stm_c1.xml', 'A100_02385_04_stm_c1.xml', 'A100_01571_06_gu_c1.xml', 'A100_02524_20_brs_c1.xml', 'A100_02152_08_gu_c1.xml', 'A100_01133_01_gu_c1.xml', 'A100_01855_01_en_c1.xml', 'A100_00785_06_fem_c1.xml', 'A100_02408_03_brs_c1.xml', 'A100_02328_03_stm_c1.xml', 'A100_01186_16_gu_c1.xml', 'A100_01238_02_en_c1.xml', 'A100_01205_14_en_c1.xml', 'A100_00206_06_stm_c1.xml', 'A100_01616_09_gu_c1.xml', 'A100_00742_03_col_c1.xml', 'A100_02127_04_gu_c1.xml', 'A100_02152_05_gu_c1.xml', 'A100_02006_07_gu_c1.xml', 'A100_01202_02_en_c1.xml', 'A100_01517_14_stm_c1.xml', 'A100_02368_08_stm_c1.xml', 'A100_00483_01_brs_c1.xml', 'A100_00220_05_stm_c1.xml', 'A100_01006_05_col_c1.xml', 'A100_02385_01_stm_c1.xml', 'A100_02094_01_gu_c1.xml', 'A100_01498_12_gu_c1.xml', 'A100_02492_07_brs_c1.xml', 'A100_02492_11_brs_c1.xml', 'A100_02011_10_gu_c1.xml', 'A100_02401_28_brs_c1.xml', 'A100_02413_05_brs_c1.xml', 'A100_01299_13_stm_c1.xml', 'A100_02127_08_gu_c1.xml', 'A100_00124_01_stm_c1.xml', 'A100_00023_02_stm_c1.xml', 'A100_00881_07_col_c1.xml', 'A100_03453_01_col_c1.xml', 'A100_01873_02_en_c1.xml', 'A100_01608_06_gu_c1.xml', 'A100_02317_01_stm_c1.xml', 'A100_00315_02_stm_c1.xml', 'A100_03456_01_col_c1.xml', 'A100_01871_06_en_c1.xml', 'A100_00209_10_stm_c1.xml', 'A100_02918_07_col_c1.xml', 'A100_01159_07_gu_c1.xml', 'A100_00329_01_stm_c1.xml', 'A100_02296_03_stm_c1.xml', 'A100_01926_15_stm_c1.xml', 'A100_02413_09_brs_c1.xml', 'A100_02877_04_col_c1.xml', 'A100_00420_04_stm_c1.xml', 'A100_01771_05_en_c1.xml', 'A100_02152_04_gu_c1.xml', 'A100_02120_04_gu_c1.xml', 'A100_00206_07_stm_c1.xml', 'A100_00099_02_stm_c1.xml', 'A100_01517_09_stm_c1.xml', 'A100_02467_07_brs_c1.xml', 'A100_01842_02_en_c1.xml', 'A100_01582_02_gu_c1.xml', 'A100_02096_01_gu_c1.xml', 'A100_01498_11_gu_c1.xml', 'A100_00456_01_brs_c1.xml', 'A100_01564_05_gu_c1.xml', 'A100_02094_02_gu_c1.xml', 'A100_02070_05_gu_c1.xml', 'A100_01159_11_gu_c1.xml', 'A100_01974_10_stm_c1.xml', 'A100_00450_01_brs_c1.xml', 'A100_02563_04_col_c1.xml', 'A100_03475_01_col_c1.xml', 'A100_00282_10_stm_c1.xml', 'A100_00871_09_en_c1.xml', 'A100_01642_48_stm_c1.xml', 'A100_02317_06_stm_c1.xml', 'A100_02572_06_col_c1.xml', 'A100_01232_01_en_c1.xml', 'A100_03478_08_col_c1.xml', 'A100_01498_06_gu_c1.xml', 'A100_00944_04_en_c1.xml', 'A100_00742_01_col_c1.xml', 'A100_01863_09_en_c1.xml', 'A100_01759_03_en_c1.xml', 'A100_02578_06_col_c1.xml', 'A100_01004_06_en_c1.xml', 'A100_01564_10_gu_c1.xml', 'A100_02467_15_brs_c1.xml', 'A100_01158_05_gu_c1.xml', 'A100_00535_06_lvr_c1.xml', 'A100_00095_01_stm_c1.xml', 'A100_00940_11_bp_c1.xml', 'A100_02385_03_stm_c1.xml', 'A100_01642_44_stm_c1.xml', 'A100_02368_03_stm_c1.xml', 'A100_02918_09_col_c1.xml', 'A100_01205_17_en_c1.xml', 'A100_00285_14_stm_c1.xml', 'A100_02099_04_gu_c1.xml', 'A100_00906_29_stm_c1.xml', 'A100_00981_04_col_c1.xml', 'A100_01192_15_gu_c1.xml', 'A100_02094_04_gu_c1.xml', 'A100_01205_22_en_c1.xml', 'A100_00298_02_stm_c1.xml', 'A100_02307_07_stm_c1.xml', 'A100_02953_05_col_c1.xml', 'A100_01031_16_stm_c1.xml', 'A100_01290_16_stm_c1.xml', 'A100_01505_03_en_c1.xml', 'A100_02421_01_brs_c1.xml', 'A100_03478_10_col_c1.xml', 'A100_02426_02_brs_c1.xml', 'A100_02413_10_brs_c1.xml', 'A100_00571_08_col_c1.xml', 'A100_02473_04_bp_c1.xml', 'A100_01606_05_gu_c1.xml', 'A100_03478_09_col_c1.xml', 'A100_02356_02_stm_c1.xml', 'A100_03416_01_col_c1.xml', 'A100_01159_15_gu_c1.xml', 'A100_00782_01_fem_c1.xml', 'A100_02467_03_brs_c1.xml', 'A100_02386_08_stm_c1.xml', 'A100_01685_01_en_c1.xml', 'A100_02152_01_gu_c1.xml', 'A100_01527_10_stm_c1.xml', 'A100_00828_20_hn_c1.xml', 'A100_01642_29_stm_c1.xml', 'A100_02575_08_col_c1.xml', 'A100_01159_01_gu_c1.xml', 'A100_01642_32_stm_c1.xml', 'A100_03416_02_col_c1.xml', 'A100_01650_16_stm_c1.xml', 'A100_01049_09_bp_c1.xml', 'A100_02099_02_gu_c1.xml', 'A100_00285_05_stm_c1.xml', 'A100_01546_06_gu_c1.xml', 'A100_00749_15_fem_c1.xml', 'A100_00828_14_hn_c1.xml', 'A100_00596_06_col_c1.xml', 'A100_02538_03_col_c1.xml', 'A100_02307_05_stm_c1.xml', 'A100_02492_02_brs_c1.xml', 'A100_01751_03_en_c1.xml', 'A100_00455_01_brs_c1.xml', 'A100_02127_05_gu_c1.xml', 'A100_00728_01_col_c1.xml', 'A100_00828_16_hn_c1.xml', 'A100_01717_04_en_c1.xml', 'A100_01842_05_en_c1.xml', 'A100_00742_05_col_c1.xml', 'A100_02314_01_stm_c1.xml', 'A100_01797_02_en_c1.xml', 'A100_01006_08_col_c1.xml', 'A100_00517_08_lvr_c1.xml', 'A100_02026_01_gu_c1.xml', 'A100_01414_07_en_c1.xml', 'A100_02099_14_gu_c1.xml', 'A100_00940_13_bp_c1.xml', 'A100_02011_03_gu_c1.xml', 'A100_01238_01_en_c1.xml', 'A100_01454_06_col_c1.xml', 'A100_02301_14_stm_c1.xml', 'A100_02492_05_brs_c1.xml', 'A100_02400_03_brs_c1.xml', 'A100_02401_10_brs_c1.xml', 'A100_01582_01_gu_c1.xml', 'A100_00875_12_col_c1.xml', 'A100_01006_04_col_c1.xml', 'A100_02301_05_stm_c1.xml', 'A100_00582_05_col_c1.xml', 'A100_00875_10_col_c1.xml', 'A100_01597_06_gu_c1.xml', 'A100_02524_18_brs_c1.xml', 'A100_01420_09_col_c1.xml', 'A100_00171_01_stm_c1.xml', 'A100_00285_08_stm_c1.xml', 'A100_02149_04_gu_c1.xml', 'A100_00725_02_col_c1.xml', 'A100_02317_09_stm_c1.xml', 'A100_02575_11_col_c1.xml', 'A100_02065_01_gu_c1.xml', 'A100_02127_09_gu_c1.xml', 'A100_00475_02_brs_c1.xml', 'A100_02401_27_brs_c1.xml', 'A100_01791_02_en_c1.xml', 'A100_00940_15_bp_c1.xml', 'A100_01926_18_stm_c1.xml', 'A100_00940_21_bp_c1.xml', 'A100_00461_01_brs_c1.xml', 'A100_01159_13_gu_c1.xml', 'A100_02368_07_stm_c1.xml', 'A100_01725_01_en_c1.xml', 'A100_01883_01_en_c1.xml', 'A100_02953_09_col_c1.xml', 'A100_02524_26_brs_c1.xml', 'A100_01608_11_gu_c1.xml', 'A100_01159_17_gu_c1.xml', 'A100_00535_07_lvr_c1.xml', 'A100_01158_01_gu_c1.xml', 'A100_00828_17_hn_c1.xml', 'A100_01159_23_gu_c1.xml', 'A100_00285_13_stm_c1.xml', 'A100_02317_05_stm_c1.xml', 'A100_01630_01_gu_c1.xml', 'A100_01813_03_en_c1.xml', 'A100_01149_02_gu_c1.xml', 'A100_00749_17_fem_c1.xml', 'A100_02383_01_stm_c1.xml', 'A100_02368_01_stm_c1.xml', 'A100_01642_51_stm_c1.xml', 'A100_02070_03_gu_c1.xml', 'A100_01414_04_en_c1.xml', 'A100_01498_05_gu_c1.xml', 'A100_03478_07_col_c1.xml', 'A100_00881_04_col_c1.xml', 'A100_01570_03_gu_c1.xml', 'A100_00945_05_col_c1.xml', 'A100_01597_11_gu_c1.xml', 'A100_00074_01_stm_c1.xml', 'A100_01571_02_gu_c1.xml', 'A100_02158_02_stm_c1.xml', 'A100_00691_01_col_c1.xml', 'A100_01535_03_stm_c1.xml', 'A100_01726_01_en_c1.xml', 'A100_00405_02_stm_c1.xml', 'A100_01842_08_en_c1.xml', 'A100_01454_04_col_c1.xml', 'A100_02558_07_col_c1.xml', 'A100_02006_06_gu_c1.xml', 'A100_02360_01_stm_c1.xml', 'A100_01643_11_stm_c1.xml', 'A100_00583_07_col_c1.xml', 'A100_01147_06_gu_c1.xml', 'A100_01771_02_en_c1.xml', 'A100_01513_15_stm_c1.xml', 'A100_02426_05_brs_c1.xml', 'A100_02094_05_gu_c1.xml', 'A100_01575_05_gu_c1.xml', 'A100_01575_08_gu_c1.xml', 'A100_01642_28_stm_c1.xml', 'A100_01202_05_en_c1.xml', 'A100_01820_02_en_c1.xml', 'A100_02524_10_brs_c1.xml', 'A100_02149_01_gu_c1.xml', 'A100_00808_02_en_c1.xml', 'A100_00384_17_stm_c1.xml', 'A100_02114_02_gu_c1.xml', 'A100_02294_03_stm_c1.xml', 'A100_02506_03_brs_c1.xml', 'A100_02572_04_col_c1.xml', 'A100_02307_06_stm_c1.xml', 'A100_00094_02_stm_c1.xml', 'A100_00981_05_col_c1.xml', 'A100_01031_18_stm_c1.xml', 'A100_00456_02_brs_c1.xml', 'A100_02385_07_stm_c1.xml', 'A100_00403_01_stm_c1.xml', 'A100_01158_06_gu_c1.xml', 'A100_01505_02_en_c1.xml', 'A100_00202_07_stm_c1.xml', 'A100_01275_16_stm_c1.xml', 'A100_02114_08_gu_c1.xml', 'A100_01546_02_gu_c1.xml', 'A100_00981_07_col_c1.xml', 'A100_01582_06_gu_c1.xml', 'A100_00191_01_stm_c1.xml', 'A100_01192_11_gu_c1.xml', 'A100_01186_18_gu_c1.xml', 'A100_02149_02_gu_c1.xml', 'A100_01387_07_en_c1.xml', 'A100_02953_13_col_c1.xml', 'A100_01871_05_en_c1.xml', 'A100_02368_04_stm_c1.xml', 'A100_02360_08_stm_c1.xml', 'A100_00933_14_hn_c1.xml', 'A100_02394_02_brs_c1.xml', 'A100_01608_02_gu_c1.xml', 'A100_02099_03_gu_c1.xml', 'A100_02024_01_gu_c1.xml', 'A100_02114_05_gu_c1.xml', 'A100_00450_02_brs_c1.xml', 'A100_00568_04_col_c1.xml', 'A100_01630_02_gu_c1.xml', 'A100_02116_03_gu_c1.xml', 'A100_00282_13_stm_c1.xml', 'A100_01797_01_en_c1.xml', 'A100_00271_03_stm_c1.xml', 'A100_02360_06_stm_c1.xml', 'A100_02120_08_gu_c1.xml', 'A100_00742_06_col_c1.xml', 'A100_02455_21_bp_c1.xml', 'A100_02467_05_brs_c1.xml', 'A100_02301_22_stm_c1.xml', 'A100_02099_09_gu_c1.xml', 'A100_02538_06_col_c1.xml', 'A100_02498_06_brs_c1.xml', 'A100_02045_07_gu_c1.xml', 'A100_01186_13_gu_c1.xml', 'A100_01759_01_en_c1.xml', 'A100_01642_39_stm_c1.xml', 'A100_01771_01_en_c1.xml', 'A100_02563_07_col_c1.xml', 'A100_01090_01_sk_c1.xml', 'A100_00397_01_stm_c1.xml', 'A100_00727_01_col_c1.xml', 'A100_01570_02_gu_c1.xml', 'A100_00908_04_en_c1.xml', 'A100_03451_03_col_c1.xml', 'A100_01926_16_stm_c1.xml', 'A100_00530_05_lvr_c1.xml', 'A100_00891_19_stm_c1.xml', 'A100_01564_08_gu_c1.xml', 'A100_00220_03_stm_c1.xml', 'A100_02301_17_stm_c1.xml', 'A100_02094_03_gu_c1.xml', 'A100_01159_16_gu_c1.xml', 'A100_00555_06_col_c1.xml', 'A100_01564_06_gu_c1.xml', 'A100_02520_01_brs_c1.xml', 'A100_01863_10_en_c1.xml', 'A100_02428_01_brs_c1.xml', 'A100_00205_03_stm_c1.xml', 'A100_02126_03_gu_c1.xml', 'A100_02385_06_stm_c1.xml', 'A100_01877_02_en_c1.xml', 'A100_02152_09_gu_c1.xml', 'A100_02045_01_gu_c1.xml', 'A100_02926_04_col_c1.xml', 'A100_02055_03_gu_c1.xml', 'A100_01071_01_hn_c1.xml', 'A100_01836_02_en_c1.xml', 'A100_01686_04_en_c1.xml', 'A100_00164_01_stm_c1.xml', 'A100_02401_19_brs_c1.xml', 'A100_01268_04_stm_c1.xml', 'A100_00087_01_stm_c1.xml', 'A100_01582_03_gu_c1.xml', 'A100_03456_03_col_c1.xml', 'A100_00535_09_lvr_c1.xml', 'A100_01774_03_en_c1.xml', 'A100_00483_02_brs_c1.xml', 'A100_01766_04_en_c1.xml', 'A100_02524_07_brs_c1.xml', 'A100_01420_08_col_c1.xml', 'A100_01275_24_stm_c1.xml', 'A100_00220_01_stm_c1.xml', 'A100_00581_04_col_c1.xml', 'A100_01279_19_stm_c1.xml', 'A100_00566_03_col_c1.xml', 'A100_01631_02_gu_c1.xml', 'A100_02152_07_gu_c1.xml', 'A100_01006_06_col_c1.xml', 'A100_00809_02_hn_c1.xml', 'A100_03416_03_col_c1.xml', 'A100_02314_02_stm_c1.xml', 'A100_00090_01_stm_c1.xml', 'A100_01773_01_en_c1.xml', 'A100_02401_09_brs_c1.xml', 'A100_02156_01_stm_c1.xml', 'A100_00220_04_stm_c1.xml', 'A100_02307_03_stm_c1.xml', 'A100_01472_02_en_c1.xml', 'A100_02365_03_stm_c1.xml', 'A100_00875_06_col_c1.xml', 'A100_02475_10_bp_c1.xml', 'A100_01080_03_hn_c1.xml', 'A100_01836_04_en_c1.xml', 'A100_02386_03_stm_c1.xml', 'A100_01658_26_stm_c1.xml', 'A100_02918_05_col_c1.xml', 'A100_01842_04_en_c1.xml', 'A100_03457_01_col_c1.xml', 'A100_02953_04_col_c1.xml', 'A100_02425_01_brs_c1.xml', 'A100_01863_03_en_c1.xml', 'A100_02401_17_brs_c1.xml', 'A100_02073_03_gu_c1.xml', 'A100_02139_08_gu_c1.xml', 'A100_02524_22_brs_c1.xml', 'A100_02308_10_stm_c1.xml', 'A100_03442_03_col_c1.xml', 'A100_00944_08_en_c1.xml', 'A100_02483_02_bp_c1.xml', 'A100_01597_04_gu_c1.xml', 'A100_00318_05_stm_c1.xml', 'A100_00277_02_stm_c1.xml', 'A100_00416_02_stm_c1.xml', 'A100_01713_04_en_c1.xml', 'A100_02413_18_brs_c1.xml', 'A100_00926_26_stm_c1.xml', 'A100_00197_01_stm_c1.xml', 'A100_02317_10_stm_c1.xml', 'A100_02386_07_stm_c1.xml', 'A100_01414_05_en_c1.xml', 'A100_02953_10_col_c1.xml', 'A100_02386_01_stm_c1.xml', 'A100_02953_11_col_c1.xml', 'A100_02455_14_bp_c1.xml', 'A100_00770_01_fem_c1.xml', 'A100_02023_04_gu_c1.xml', 'A100_01186_15_gu_c1.xml', 'A100_02467_09_brs_c1.xml', 'A100_00023_01_stm_c1.xml', 'A100_02301_07_stm_c1.xml', 'A100_01694_01_en_c1.xml', 'A100_01560_05_gu_c1.xml', 'A100_01642_52_stm_c1.xml', 'A100_01571_05_gu_c1.xml', 'A100_01926_13_stm_c1.xml', 'A100_00090_02_stm_c1.xml', 'A100_02575_10_col_c1.xml', 'A100_00079_01_stm_c1.xml', 'A100_01642_23_stm_c1.xml', 'A100_02317_11_stm_c1.xml', 'A100_02527_01_brs_c1.xml', 'A100_02385_10_stm_c1.xml', 'A100_00420_01_stm_c1.xml', 'A100_01597_09_gu_c1.xml', 'A100_03442_04_col_c1.xml', 'A100_02413_11_brs_c1.xml', 'A100_00285_07_stm_c1.xml', 'A100_03041_01_col_c1.xml', 'A100_00315_04_stm_c1.xml', 'A100_02490_01_bp_c1.xml', 'A100_01613_08_gu_c1.xml', 'A100_01773_09_en_c1.xml', 'A100_00765_03_fem_c1.xml', 'A100_02386_06_stm_c1.xml', 'A100_02317_04_stm_c1.xml', 'A100_00008_01_stm_c1.xml', 'A100_01270_09_stm_c1.xml', 'A100_01597_02_gu_c1.xml', 'A100_03485_02_col_c1.xml', 'A100_00456_03_brs_c1.xml', 'A100_01014_06_en_c1.xml', 'A100_01576_05_gu_c1.xml', 'A100_01747_08_en_c1.xml', 'A100_01795_02_en_c1.xml', 'A100_01642_50_stm_c1.xml', 'A100_00597_11_bp_c1.xml', 'A100_00828_22_hn_c1.xml', 'A100_02467_13_brs_c1.xml', 'A100_00597_13_bp_c1.xml', 'A100_02296_06_stm_c1.xml', 'A100_01643_12_stm_c1.xml', 'A100_01498_10_gu_c1.xml', 'A100_00929_07_en_c1.xml', 'A100_02413_20_brs_c1.xml', 'A100_01642_12_stm_c1.xml', 'A100_00935_07_en_c1.xml', 'A100_01089_01_sk_c1.xml', 'A100_01517_13_stm_c1.xml', 'A100_02400_02_brs_c1.xml', 'A100_00591_05_col_c1.xml', 'A100_00981_09_col_c1.xml', 'A100_01643_17_stm_c1.xml', 'A100_02120_03_gu_c1.xml', 'A100_01730_04_en_c1.xml', 'A100_03478_04_col_c1.xml', 'A100_01616_02_gu_c1.xml', 'A100_02360_07_stm_c1.xml', 'A100_02385_08_stm_c1.xml', 'A100_00785_07_fem_c1.xml', 'A100_01158_04_gu_c1.xml', 'A100_02385_11_stm_c1.xml', 'A100_00945_06_col_c1.xml', 'A100_00938_05_en_c1.xml', 'A100_02524_05_brs_c1.xml', 'A100_00202_05_stm_c1.xml', 'A100_02524_16_brs_c1.xml', 'A100_01044_07_bp_c1.xml', 'A100_02152_03_gu_c1.xml', 'A100_01642_14_stm_c1.xml', 'A100_01046_11_bp_c1.xml', 'A100_03451_04_col_c1.xml', 'A100_00309_04_stm_c1.xml', 'A100_02575_09_col_c1.xml', 'A100_01159_21_gu_c1.xml', 'A100_02524_01_brs_c1.xml', 'A100_01725_02_en_c1.xml', 'A100_02049_02_gu_c1.xml', 'A100_03457_04_col_c1.xml', 'A100_00785_04_fem_c1.xml', 'A100_01118_01_gu_c1.xml', 'A100_00785_08_fem_c1.xml', 'A100_02296_08_stm_c1.xml', 'A100_00935_09_en_c1.xml', 'A100_01602_05_gu_c1.xml', 'A100_02575_04_col_c1.xml', 'A100_02360_05_stm_c1.xml', 'A100_02001_02_gu_c1.xml', 'A100_01650_17_stm_c1.xml', 'A100_02317_12_stm_c1.xml', 'A100_01631_03_gu_c1.xml', 'A100_00749_18_fem_c1.xml', 'A100_02575_06_col_c1.xml', 'A100_01630_03_gu_c1.xml', 'A100_02538_04_col_c1.xml', 'A100_02070_04_gu_c1.xml', 'A100_01926_12_stm_c1.xml', 'A100_02368_10_stm_c1.xml', 'A100_01597_05_gu_c1.xml', 'A100_00420_03_stm_c1.xml', 'A100_00318_07_stm_c1.xml', 'A100_00571_07_col_c1.xml', 'A100_02385_15_stm_c1.xml', 'A100_02301_16_stm_c1.xml', 'A100_02575_05_col_c1.xml', 'A100_00751_09_fem_c1.xml', 'A100_01674_02_en_c1.xml', 'A100_01774_01_en_c1.xml', 'A100_00472_01_brs_c1.xml', 'A100_01685_02_en_c1.xml', 'A100_01836_03_en_c1.xml', 'A100_02365_01_stm_c1.xml', 'A100_02575_07_col_c1.xml', 'A100_01642_27_stm_c1.xml', 'A100_01299_15_stm_c1.xml', 'A100_01123_02_gu_c1.xml', 'A100_02120_09_gu_c1.xml', 'A100_01275_20_stm_c1.xml', 'A100_01616_12_gu_c1.xml', 'A100_01454_03_col_c1.xml', 'A100_00205_04_stm_c1.xml', 'A100_01771_03_en_c1.xml', 'A100_02360_10_stm_c1.xml', 'A100_02114_04_gu_c1.xml', 'A100_02280_03_stm_c1.xml', 'A100_01248_01_en_c1.xml', 'A100_00934_26_sk_c1.xml', 'A100_02336_04_stm_c1.xml', 'A100_02483_03_bp_c1.xml', 'A100_01555_12_gu_c1.xml', 'A100_01571_07_gu_c1.xml', 'A100_02084_01_gu_c1.xml', 'A100_02049_01_gu_c1.xml', 'A100_02492_03_brs_c1.xml', 'A100_01579_13_gu_c1.xml', 'A100_01205_12_en_c1.xml', 'A100_01560_07_gu_c1.xml', 'A100_01737_03_en_c1.xml', 'A100_02149_03_gu_c1.xml', 'A100_01265_08_stm_c1.xml', 'A100_01544_04_gu_c1.xml', 'A100_00271_01_stm_c1.xml', 'A100_01842_03_en_c1.xml', 'A100_02307_04_stm_c1.xml', 'A100_02401_03_brs_c1.xml', 'A100_01514_07_stm_c1.xml', 'A100_00583_04_col_c1.xml', 'A100_01555_09_gu_c1.xml', 'A100_01565_02_gu_c1.xml', 'A100_02467_12_brs_c1.xml', 'A100_00461_02_brs_c1.xml', 'A100_02314_04_stm_c1.xml', 'A100_00420_02_stm_c1.xml', 'A100_02524_06_brs_c1.xml', 'A100_02307_13_stm_c1.xml', 'A100_01642_37_stm_c1.xml', 'A100_01642_43_stm_c1.xml', 'A100_02385_13_stm_c1.xml', 'A100_03478_11_col_c1.xml', 'A100_02290_01_stm_c1.xml', 'A100_02385_09_stm_c1.xml', 'A100_03475_03_col_c1.xml', 'A100_02294_01_stm_c1.xml', 'A100_00846_06_en_c1.xml', 'A100_02524_17_brs_c1.xml', 'A100_01894_01_en_c1.xml', 'A100_01726_02_en_c1.xml', 'A100_02578_05_col_c1.xml', 'A100_02114_07_gu_c1.xml', 'A100_01642_15_stm_c1.xml', 'A100_00742_04_col_c1.xml', 'A100_01420_05_col_c1.xml', 'A100_01546_05_gu_c1.xml', 'A100_02368_02_stm_c1.xml', 'A100_02426_03_brs_c1.xml', 'A100_00099_01_stm_c1.xml', 'A100_02099_13_gu_c1.xml', 'A100_01265_07_stm_c1.xml', 'A100_02572_08_col_c1.xml', 'A100_02953_07_col_c1.xml', 'A100_00282_11_stm_c1.xml', 'A100_01576_02_gu_c1.xml', 'A100_01760_02_en_c1.xml', 'A100_03478_03_col_c1.xml', 'A100_00199_03_stm_c1.xml', 'A100_01773_05_en_c1.xml', 'A100_01642_18_stm_c1.xml', 'A100_00318_03_stm_c1.xml', 'A100_01255_02_en_c1.xml', 'A100_01842_07_en_c1.xml', 'A100_02483_04_bp_c1.xml', 'A100_02926_05_col_c1.xml', 'A100_00571_03_col_c1.xml', 'A100_00202_06_stm_c1.xml', 'A100_00416_01_stm_c1.xml', 'A100_00199_04_stm_c1.xml', 'A100_02098_01_gu_c1.xml', 'A100_01387_08_en_c1.xml', 'A100_02413_13_brs_c1.xml', 'A100_01616_06_gu_c1.xml', 'A100_02467_06_brs_c1.xml', 'A100_02283_03_stm_c1.xml', 'A100_01486_10_en_c1.xml', 'A100_00597_10_bp_c1.xml', 'A100_00271_02_stm_c1.xml', 'A100_01747_03_en_c1.xml', 'A100_00318_12_stm_c1.xml', 'A100_00149_01_stm_c1.xml', 'A100_01836_05_en_c1.xml', 'A100_00751_12_fem_c1.xml', 'A100_02538_05_col_c1.xml', 'A100_01885_02_en_c1.xml', 'A100_01268_07_stm_c1.xml', 'A100_02336_02_stm_c1.xml', 'A100_02498_01_brs_c1.xml', 'A100_02026_04_gu_c1.xml', 'A100_01774_02_en_c1.xml', 'A100_02360_03_stm_c1.xml', 'A100_01861_03_en_c1.xml', 'A100_02314_03_stm_c1.xml', 'A100_01165_02_gu_c1.xml', 'A100_00408_01_stm_c1.xml', 'A100_01643_15_stm_c1.xml', 'A100_02139_06_gu_c1.xml', 'A100_00575_09_col_c1.xml', 'A100_01846_03_en_c1.xml', 'A100_02127_07_gu_c1.xml', 'A100_02473_03_bp_c1.xml', 'A100_02114_06_gu_c1.xml', 'A100_02114_03_gu_c1.xml', 'A100_02363_01_stm_c1.xml', 'A100_01232_02_en_c1.xml', 'A100_02099_08_gu_c1.xml', 'A100_02518_02_brs_c1.xml', 'A100_02073_02_gu_c1.xml', 'A100_02356_04_stm_c1.xml', 'A100_02527_07_brs_c1.xml', 'A100_01299_14_stm_c1.xml', 'A100_02127_12_gu_c1.xml', 'A100_01498_13_gu_c1.xml', 'A100_01873_01_en_c1.xml', 'A100_01544_07_gu_c1.xml', 'A100_03457_02_col_c1.xml', 'A100_01597_08_gu_c1.xml', 'A100_01374_11_col_c1.xml', 'A100_02301_19_stm_c1.xml', 'A100_03485_04_col_c1.xml', 'A100_01159_20_gu_c1.xml', 'A100_02385_02_stm_c1.xml', 'A100_02112_02_gu_c1.xml', 'A100_01795_04_en_c1.xml', 'A100_02152_02_gu_c1.xml', 'A100_00323_02_stm_c1.xml', 'A100_01159_18_gu_c1.xml', 'A100_00597_09_bp_c1.xml', 'A100_01642_34_stm_c1.xml', 'A100_02383_02_stm_c1.xml', 'A100_01159_06_gu_c1.xml', 'A100_02041_04_gu_c1.xml', 'A100_02116_01_gu_c1.xml', 'A100_00530_02_lvr_c1.xml', 'A100_02070_01_gu_c1.xml', 'A100_02563_06_col_c1.xml', 'A100_01159_12_gu_c1.xml', 'A100_02531_02_brs_c1.xml', 'A100_02127_11_gu_c1.xml', 'A100_00981_06_col_c1.xml', 'A100_01054_21_bp_c1.xml', 'A100_02369_01_stm_c1.xml', 'A100_02368_11_stm_c1.xml', 'A100_01597_03_gu_c1.xml', 'A100_01414_06_en_c1.xml', 'A100_01861_02_en_c1.xml', 'A100_03453_04_col_c1.xml', 'A100_02006_09_gu_c1.xml', 'A100_00398_01_stm_c1.xml', 'A100_00318_15_stm_c1.xml', 'A100_02953_12_col_c1.xml', 'A100_01795_01_en_c1.xml', 'A100_00751_10_fem_c1.xml', 'A100_01244_01_en_c1.xml', 'A100_02360_02_stm_c1.xml', 'A100_01793_02_en_c1.xml', 'A100_02953_03_col_c1.xml', 'A100_02317_02_stm_c1.xml', 'A100_00078_01_stm_c1.xml', 'A100_02953_06_col_c1.xml', 'A100_01498_14_gu_c1.xml', 'A100_00581_05_col_c1.xml', 'A100_01159_08_gu_c1.xml', 'A100_00785_02_fem_c1.xml', 'A100_03442_06_col_c1.xml', 'A100_03485_03_col_c1.xml', 'A100_00117_01_stm_c1.xml', 'A100_02524_03_brs_c1.xml', 'A100_02296_09_stm_c1.xml', 'A100_00323_03_stm_c1.xml', 'A100_00945_07_col_c1.xml', 'A100_02096_07_gu_c1.xml', 'A100_02114_01_gu_c1.xml', 'A100_02524_23_brs_c1.xml', 'A100_00881_05_col_c1.xml', 'A100_01643_14_stm_c1.xml', 'A100_02401_16_brs_c1.xml', 'A100_01750_02_en_c1.xml', 'A100_02054_05_gu_c1.xml', 'A100_00981_08_col_c1.xml', 'A100_02045_03_gu_c1.xml', 'A100_02572_05_col_c1.xml', 'A100_01546_04_gu_c1.xml', 'A100_01730_02_en_c1.xml', 'A100_02073_04_gu_c1.xml', 'A100_02011_19_gu_c1.xml', 'A100_01608_09_gu_c1.xml', 'A100_00206_04_stm_c1.xml', 'A100_01600_06_gu_c1.xml', 'A100_03478_02_col_c1.xml', 'A100_01597_10_gu_c1.xml', 'A100_01766_06_en_c1.xml', 'A100_02065_13_gu_c1.xml', 'A100_02926_03_col_c1.xml', 'A100_00597_15_bp_c1.xml', 'A100_02317_03_stm_c1.xml', 'A100_01454_05_col_c1.xml', 'A100_00548_06_col_c1.xml', 'A100_02953_14_col_c1.xml', 'A100_01374_07_col_c1.xml', 'A100_00940_16_bp_c1.xml', 'A100_00715_01_col_c1.xml', 'A100_00159_01_stm_c1.xml', 'A100_00008_02_stm_c1.xml', 'A100_01773_06_en_c1.xml', 'A100_02308_09_stm_c1.xml', 'A100_02483_01_bp_c1.xml', 'A100_03475_04_col_c1.xml', 'A100_02024_04_gu_c1.xml', 'A100_01613_05_gu_c1.xml', 'A100_01575_01_gu_c1.xml', 'A100_00209_03_stm_c1.xml', 'A100_02023_02_gu_c1.xml', 'A100_01766_01_en_c1.xml', 'A100_00318_02_stm_c1.xml', 'A100_02308_01_stm_c1.xml', 'A100_02386_05_stm_c1.xml', 'A100_02558_05_col_c1.xml', 'A100_02120_05_gu_c1.xml', 'A100_00940_20_bp_c1.xml', 'A100_00164_02_stm_c1.xml', 'A100_00309_06_stm_c1.xml', 'A100_01546_03_gu_c1.xml', 'A100_01642_31_stm_c1.xml', 'A100_01791_01_en_c1.xml', 'A100_00749_20_fem_c1.xml', 'A100_01579_02_gu_c1.xml', 'A100_00346_01_stm_c1.xml', 'A100_03442_01_col_c1.xml', 'A100_02301_13_stm_c1.xml', 'A100_01159_22_gu_c1.xml', 'A100_02061_02_gu_c1.xml', 'A100_02099_01_gu_c1.xml', 'A100_02314_05_stm_c1.xml', 'A100_01560_02_gu_c1.xml', 'A100_00079_02_stm_c1.xml', 'A100_01586_05_gu_c1.xml', 'A100_03442_02_col_c1.xml', 'A100_01571_04_gu_c1.xml', 'A100_00101_01_stm_c1.xml', 'A100_00575_12_col_c1.xml', 'A100_02524_13_brs_c1.xml', 'A100_00828_23_hn_c1.xml', 'A100_01643_16_stm_c1.xml', 'A100_01046_08_bp_c1.xml', 'A100_02413_06_brs_c1.xml', 'A100_01555_02_gu_c1.xml', 'A100_02139_05_gu_c1.xml', 'A100_01420_07_col_c1.xml', 'A100_02054_02_gu_c1.xml', 'A100_02096_06_gu_c1.xml', 'A100_02099_05_gu_c1.xml', 'A100_01717_01_en_c1.xml', 'A100_01597_07_gu_c1.xml', 'A100_00506_01_brs_c1.xml', 'A100_02524_09_brs_c1.xml', 'A100_02127_03_gu_c1.xml', 'A100_02421_02_brs_c1.xml', 'A100_00530_04_lvr_c1.xml', 'A100_01255_01_en_c1.xml', 'A100_00220_02_stm_c1.xml', 'A100_00734_01_col_c1.xml', 'A100_02572_07_col_c1.xml', 'A100_00846_08_en_c1.xml', 'A100_01855_02_en_c1.xml', 'A100_00020_01_stm_c1.xml', 'A100_00517_06_lvr_c1.xml', 'A100_01159_02_gu_c1.xml', 'A100_02152_11_gu_c1.xml', 'A100_00785_05_fem_c1.xml', 'A100_02099_06_gu_c1.xml', 'A100_01694_02_en_c1.xml', 'A100_01631_05_gu_c1.xml', 'A100_02538_07_col_c1.xml', 'A100_01795_05_en_c1.xml']

In [70]:
print(len(name_list))
print(len(in_list))

1173
1165


In [72]:
for i in range(len(name_list)):
    p = name_list[i]
    p_x = p +'.xml'
    if p_x not in in_list:
        print("'"+p+"',")

'A100_01044_05_bp_c1',
'A100_01044_06_bp_c1',
'A100_02127_01_gu_c1',
'A100_02524_02_brs_c1',
'A100_02575_12_col_c1',
'A100_02578_02_col_c1',
'A100_02578_03_col_c1',
'A100_02578_04_col_c1',


In [77]:
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,100).__str__()

In [86]:
name_list = [
'A100_02575_12_col_c1','A100_02578_02_col_c1','A100_02578_03_col_c1','A100_02578_04_col_c1']

In [88]:
png_5_l = os.listdir('/workspace/src/chaehyeon/png_contour/xml_smooth/5X')
print(len(png_5_l))

1202


In [ ]:
'A100_01044_05_bp_c1', 'A100_01044_06_bp_c1', 'A100_02127_01_gu_c1', 'A100_02524_02_brs_c1',